# Общая информация
## Как подключиться к базе данных
### Как подключиться к PostgreSQL через psycopg2
1. Импортируем библиотеку:

In [2]:
import psycopg2

2. Далее используем данные для входа

In [3]:
database="startml",
user="robot-startml-ro",
password="pheiph0hahj1Vaif",
host="postgres.lab.karpov.courses",
port=6432

### Как подключиться к PostgreSQL через pandas
Для загрузки таблицы используется функция pd.read_sql.

In [4]:
import pandas as pd

df = pd.read_sql(
    """SELECT * FROM "feed_action" LIMIT 10 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df.head()

,user_id,post_id,action,time
0,121060,1377,like,2021-11-08 15:19:31
1,123550,1447,view,2021-11-08 15:19:31
2,109562,1301,view,2021-11-08 15:19:31
3,11199,1389,view,2021-11-08 15:19:32
4,110435,1466,view,2021-11-08 15:19:32


Обратите внимание, что сам запрос выделяется тройными кавычками (""") — этот знак в Python для многострочного ввода. Таким же образом выделяются многострочные комментарии в коде.

Мы будем работать с базой данных, в которой имеется три таблицы:

1. Таблица с данными о пользователях (user):

id - уникальный идентификатор пользователя (primary key)

gender - пол

age - возраст

country - страна

city - город

exp_group - экспериментальная группа

os - операционная система

source - источник трафика

2. Таблица с данными о постах (post):

id - уникальный идентификатор поста (primary key)

text - текст поста

topic - тема поста

3. Таблица с данными о действиях пользователей (feed_action):

user_id (——>) user (id)- идентификатор пользователя     

post_id (——>) post (id)- идентификатор поста     

action - совершенное в сети действие     

time - время действия

1.1. Сделайте запрос на все колонки из таблицы user. Для этого может понадобиться взять user в кавычки, вот так: "user" - чтобы PostgreSQL не спутал ее со своей встроенной таблицей user, где он ведет учет всех пользователей.

In [5]:
df_user = pd.read_sql(
    '''SELECT * FROM "user" LIMIT 10 ''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_user.head()

,id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


1.2. Отберите уникальные значения тем (topic) в таблице постов (post).

In [6]:
df_unique = pd.read_sql(
    '''SELECT DISTINCT(topic) FROM "post"''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_unique

,topic
0,tech
1,covid
2,movie
3,entertainment
4,politics
5,sport
6,business


2.1. Отберите пользователей старше 30 с устройством на iOS. Каков ID и город записи с минимальным ID (он идет первым в таблице)?

In [8]:
df_slice1 = pd.read_sql(
    '''SELECT id, city FROM "user"
    WHERE age > 30 AND os = 'iOS'
    ORDER BY id''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_slice1

,id,city
0,212,Podolsk
1,222,Ulyanovsk
2,224,Yekaterinburg
3,226,Moscow
4,243,Ekibastuz
...,...,...
17665,168517,Urzhum
17666,168525,Pavlohrad
17667,168536,Michurinsk
17668,168540,Tukan


In [ ]:
2.2. Отберите пользователей, которые не из России и при этом либо их экспериментальная группа не 0 и не 3 или их город – Минск.

In [10]:
df_slice2 = pd.read_sql(
    '''SELECT * FROM "user"
    WHERE country != 'Russia' AND ((exp_group != 0 AND exp_group != 3) OR city = 'Minsk')
    ORDER BY id''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_slice2

,id,gender,age,country,city,exp_group,os,source
0,211,1,22,Belarus,Minsk,0,iOS,ads
1,213,1,22,Ukraine,Kamianske,2,iOS,ads
2,216,0,38,Ukraine,Lviv,1,Android,ads
3,218,1,25,Belarus,Minsk,2,iOS,ads
4,228,0,17,Azerbaijan,Baku,2,iOS,ads
...,...,...,...,...,...,...,...,...
12471,168477,1,20,Turkey,Ankara,4,Android,organic
12472,168524,1,17,Turkey,Ereğli,2,Android,organic
12473,168525,1,39,Ukraine,Pavlohrad,1,iOS,organic
12474,168527,1,20,Ukraine,Sumy,2,iOS,organic


In [ ]:
3.1. Подсчитайте средний возраст пользователей в разрезе страны. В ответ напишите средний возраст пользователей из Кипра. Ответ округлите до двух знаков после точки.

In [12]:
df_group1 = pd.read_sql(
    '''SELECT country, ROUND(AVG(age), 2) FROM "user"
    GROUP BY country''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_group1

,country,round
0,Azerbaijan,26.78
1,Belarus,27.28
2,Cyprus,28.07
3,Estonia,28.15
4,Finland,27.13
5,Kazakhstan,27.22
6,Latvia,27.22
7,Russia,27.19
8,Switzerland,27.95
9,Turkey,27.45


3.2. Сгруппируйте пользователей сначала по экспериментальной группе, а затем по операционной системе. В каждой группе подсчитайте количество пользователей, максимальный и минимальный возраст.

Сохраните результаты в файл CSV с колонками в следующем порядке: exp_group, os, total_users, max_age, min_age (обратите внимание на название последних трех колонок!), используя запятую в качестве сепаратора, и отправьте файл в LMS.

In [17]:
df_group2 = pd.read_sql(
    '''SELECT exp_group, os, COUNT(id) AS total_users, MAX(age) AS max_age, MIN(age) AS min_age FROM "user"
    GROUP BY exp_group, os''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_group2

,exp_group,os,total_users,max_age,min_age
0,0,Android,21234,95,14
1,0,iOS,11489,79,14
2,1,Android,21232,92,14
3,1,iOS,11406,87,14
4,2,Android,21102,78,14
5,2,iOS,11512,74,14
6,3,Android,21319,84,14
7,3,iOS,11449,85,14
8,4,Android,21085,84,14
9,4,iOS,11377,78,14


In [18]:
df_group2.to_csv('result1.csv', sep=',', index=False)

3.3. В каждой категории постов определите длину самого длинного текста, используя таблицу post. В ответ отправьте тему, у которой длина самого большого поста больше 25 000.

In [19]:
df_group3_1 = pd.read_sql(
    '''SELECT * FROM "post"''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_group3_1.head()

,id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [21]:
df_group3_2 = pd.read_sql(
    '''SELECT topic, MAX(LENGTH(text)) FROM "post"
    GROUP BY topic''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_group3_2.head()

,topic,max
0,tech,16063
1,covid,148
2,movie,10234
3,entertainment,19004
4,politics,25392


3.4. Найдите страну с наименьшим числом пользователей среди стран, у которых общее число пользователей больше 1000.

In [26]:
df_group4 = pd.read_sql(
    '''SELECT country, COUNT(id) AS num_users FROM "user"
    GROUP BY country
    HAVING COUNT(id) > 1000
    ORDER BY num_users''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_group4.head()

,country,num_users
0,Azerbaijan,1542
1,Finland,1599
2,Turkey,1606
3,Kazakhstan,3172
4,Belarus,3293


3.5. Отберите для пользователей из Москвы экспериментальные группы, в которых средний возраст больше 27.2.

In [27]:
df_group5 = pd.read_sql(
    '''SELECT exp_group, COUNT(id) AS num_users FROM "user"
    WHERE city = 'Moscow'
    GROUP BY exp_group
    HAVING AVG(age) > 27.2''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_group5.head()

,exp_group,num_users
0,1,4414
1,3,4378


4.1. Найдите в таблице post 3 темы с наибольшим количеством постов.

In [30]:
df_sort1 = pd.read_sql(
    '''SELECT topic, count(id) FROM "post"
    GROUP BY topic
    ORDER BY count(id) DESC''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_sort1.head()

,topic,count
0,movie,3000
1,covid,1799
2,business,510
3,sport,510
4,politics,417


4.2. Отберите пользователей из Воронежа (Voronezh) и отсортируйте их по возрасту в убывающем порядке. Для равных возрастов отсортируйте по возрастанию экспериментальной группы.

Напишите ID второго в порядке выдачи пользователя.

In [32]:
df_sort2 = pd.read_sql(
    '''SELECT id, city, age, exp_group FROM "user"
    WHERE city = 'Voronezh'
    ORDER BY age DESC, exp_group ASC''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_sort2.head()

,id,city,age,exp_group
0,89990,Voronezh,71,0
1,2441,Voronezh,71,3
2,14184,Voronezh,66,3
3,35093,Voronezh,65,0
4,77284,Voronezh,64,4


5.1. Отберите post_id, time, age и os пользователя для всех лайков пользователей из Омска. Отсортируйте результаты по убыванию времени действия и загрузите в LMS первые 100 значений в формате CSV (сепаратор "запятая", колонки с теми же названиями, что и в задании).

In [34]:
df_join1 = pd.read_sql(
    '''SELECT f.post_id, f.time, u.age, u.os FROM "user" AS u
    LEFT JOIN "feed_action" f ON u.id = f.user_id
    WHERE u.city = 'Omsk' and f.action = 'like'
    ORDER BY f.time DESC
    LIMIT 100''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_join1.head()

,post_id,time,age,os
0,7159,2022-01-26 23:59:54,17,Android
1,7184,2022-01-26 23:56:30,17,Android
2,7265,2022-01-26 23:56:03,17,Android
3,7256,2022-01-26 23:55:24,17,Android
4,7080,2022-01-26 23:55:12,17,Android


In [35]:
df_join1.to_csv('result2_join.csv', sep=',', index=False)

5.2. Найдите город, пользователи которого в возрасте 36 лет 2021-12-01 числа просмотрели запись на тему covid меньше всего раз относительно других городов.

In [40]:
df_join2 = pd.read_sql(
    '''SELECT u.city, COUNT(u.id) AS num_views
    FROM "user" AS u
    LEFT JOIN "feed_action" f ON u.id = f.user_id
    LEFT JOIN "post" p ON f.post_id = p.id
    WHERE u.age = 36 AND DATE(f.time) = '2021-12-01' AND p.topic = 'covid'
    GROUP BY u.city
    ORDER BY num_views ASC''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_join2.head()

,city,num_views
0,Arkhangelsk,6
1,Fryazino,9
2,Apsheronsk,10
3,Lubny,10
4,Kirs,11


6. Напишите SQL запрос, используя который вы получите:
- топ N постов по лайкам,
- количество пролайкавших пользователей,
- время последнего лайка.

В качестве ответа напишите через пробел первые 3 ID постов из этого топа.

In [42]:
df_final = pd.read_sql(
    '''SELECT post_id, COUNT(action = 'like') AS top_liked, COUNT(user_id) AS num_likes, MAX(time)
    FROM "feed_action"
    GROUP BY post_id
    ORDER BY top_liked DESC''',
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_final.head()

,post_id,top_liked,num_likes,max
0,4866,136052,136052,2022-01-01 13:59:50
1,5127,127054,127054,2022-01-05 13:54:49
2,6448,120387,120387,2022-01-22 05:58:41
3,6498,119669,119669,2022-01-22 21:59:17
4,6513,119261,119261,2022-01-22 19:59:52
